In [32]:
import numpy as np
from collections import defaultdict
import random
from numpy import random
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%run Data_Center_setup.ipynb

from collections import defaultdict

import gurobipy as gp
from gurobipy import GRB
from gurobipy import*
from gurobipy import quicksum

$$
\begin{equation*}
\begin{array}{lll}
&\displaystyle \min_{\substack{
u^{\text{dis}}_{t},\ u^{\text{chr}}_{t},p^{\text{dis}}_{t},\ p^{\text{chr}}_{t},x^{\text{chiller}}_{j,t},\ x^{\text{tower}}_{j,t} \\
T_{i,t}^{\text{Zone}},\ T_{i,t+1}^{\text{Zone}} ,T_{i,t}^{\text{sup}},\ v^{\text{vent}}_{t} \\}} 
\sum_{t=0}^{T}p^{e,g}_{t} e^{\text{dc,in}}_{t} & \\
s.t.\quad &e^{\text{dc,in}}_{t} = E^{\text{HVAC}}_t + E^{\text{Se}}_{t} +\Delta E^{\text{B}}_{t} - E^{\text{Solar}}_{t} & \forall  t \in \mathbf{T}\\
&e^{\text{dc,in}}_{t} \geq 0 & \forall t \in \mathbf{T}\\
& \Delta E^{\text{B}}_{t} = p^{\text{chr}}_{t}\eta^{\text{chr}} - p^{\text{dis}}_{t} \cdot (\eta^{\text{dis}})^{-1} & \forall  t \in \mathbf{T}\\ 
& E^{\text{B,state}}_{t+1} = E^{\text{B,state}}_{t} + \Delta E^{\text{B}}_{t} & \forall  t \in \mathbf{T} \\
& \underline{\xi^{\text{B}}} \leq E^{\text{B,state}}_{t} \leq \overline{\xi^{\text{B}}} & \forall  t \in \mathbf{T} \\
& p^{\text{chr}}_{t} \leq \overline{p^{\text{chr}}_{t}} \cdot u^{\text{chr}}_{t} & \forall  t \in \mathbf{T}\\
& p^{\text{dis}}_{t} \leq \overline{p^{\text{dis}}_{t}} \cdot u^{\text{dis}}_{t} & \forall  t \in \mathbf{T}\\
& u^{\text{chr}}_{t} + u^{\text{dis}}_{t} \leq 1 & \forall  t \in \mathbf{T}\\
&T^{\text{Zone}}_{i,t+1} = T^{\text{Zone}}_{i,t} 
+ \displaystyle\sum_{i^{\prime} \in  \mathcal{N}\left(i\right)} 
\left(\frac{T_{i^{\prime},t}^{\text{Zone}} - T_{i,t}^{\text{Zone}}}
{C^{\text{heat}}_{i}R^{\text{Zone}}_{i^{\prime}i}}\right)
+ \frac{\dot{m}_{i} c_{a}\left(T_{i,t}^{\text{sup}} - T_{i,t}^{\text{Zone}}\right) 
+ \theta_{i,t}}{C^{\text{heat}}_{i}} & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T}\\
&T_{i,t}^{-} \leq T^{\text{Zone}}_{i,t} \leq T_{i,t}^{+}, & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T} \\
&v^{\text{vent}}_t + v^{\text{out}}_t \geq \underline{v}^{\text{vent}}_t & \forall  t \in \mathbf{T}\\
&L^{\text{heat}}_{t} = \left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{chiller}}} x^{\text{chiller}}_{j,t}m^{\text{chiller}}_{j,t} \geq \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{tower}}}{x^{\text{tower}}_{j,t}m^{\text{tower}}_{j,t}} \geq \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p }
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot  c^{\text{water}}_p }  & \forall t \in \mathbf{T}\\
&e^{\text{vent}}_{t}=\beta^{\text{vent}}_{0}+\beta^{\text{vent}}_{1}\left(v^{\text{vent}}_t-\underline{v}^{\text{vent}}\right) 
& \forall t \in \mathbf{T}\\
&e^{\text{chiller}}_{t}=\displaystyle\sum_{j \in \mathbf{J}^{\text{chiller}}}x^{\text{chiller}}_{j,t}\left(\beta^{\text{chiller}}_{0,j}
 + \beta^{\text{chiller}}_{1,j}m^{\text{chiller}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&e^{\text {pump }}_{t}=\beta^{\text{pump}}_{0}+\beta^{\text{pump}}_{1} \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&e^{\text{tower}}_{t} = \displaystyle\sum_{j \in \mathbf{J}^{\text{tower}}}x^{\text{tower}}_{j,t} \left(\beta^{\text{tower}}_{0,j} + \beta^{\text{tower}}_{1,j}m^{\text{tower}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&E^{\text{HVAC}}_{t} =  e^{\text{sup}}_{t} + e^{\text{vent}}_{t}+ e^{\text{chiller}}_{t} + e^{\text{pump}}_{t} + e^{\text{tower}}_{t} & \forall  t \in \mathbf{T}\\
\end{array}
\end{equation*}
$$

In [33]:
# create one model instance, with a name
DCMG = gp.Model("Data_center_model")

# continuous decision variables
# charging and discharging of battery rate
p_char = DCMG.addVars(T, vtype=GRB.CONTINUOUS, name='p_char')
p_dis = DCMG.addVars(T, vtype=GRB.CONTINUOUS, name='p_char')

# zone i AC temperature
Temp_AC = DCMG.addVars(I_zone, T, vtype=GRB.CONTINUOUS, name='Temp_sup')

# ventilation air flow rate
v_vent = DCMG.addVars(T, vtype=GRB.CONTINUOUS, name='v_vent')

# continuous state variables
# room air temperature
Temp_Zone = DCMG.addVars(I_zone, T + 1, vtype=GRB.CONTINUOUS, name='Temp_zone')

# battery state 
Battery_state = DCMG.addVars(T + 1, vtype=GRB.CONTINUOUS, name='Battery_state')

# eletricity demand
e_in = DCMG.addVars(T, vtype=GRB.CONTINUOUS, name='e_in')

# binary continuous variables
# charging and discharging of battery state
u_char = DCMG.addVars(T, vtype=GRB.BINARY, name='u_char')
u_dis = DCMG.addVars(T, vtype=GRB.BINARY, name='u_dis')

# chiller and tower on/off state
x_chiller = DCMG.addVars(J_chiller, T, vtype=GRB.BINARY, name='x_chiller')
x_tower = DCMG.addVars(J_tower, T, vtype=GRB.BINARY, name='x_tower')

In [34]:
# objective function
DCMG.setObjective(quicksum( e_in[t] * p_eg[t] for t in range(T)), GRB.MINIMIZE)

In [35]:
# constarint 1
for t in range(T):
    DCMG.addConstr(e_in[t] == e_server[t] + p_char[t] * eta_char_b - p_dis[t] * eta_dischar_b + e_solar[t] + e_sup[t] +\
                      beta_vent_0 + beta_vent_1 * (v_vent[t] - v_vent_startup)  +\
                      quicksum(x_chiller[j,t] * (beta_chiller_0[j] +  beta_chiller_1[j] * M_chiller_rated[j]) for j in range(J_chiller)) +\
                      quicksum(x_tower[j,t] * (beta_tower_0[j] +  beta_tower_1[j] * M_tower_rated[j]) for j in range(J_tower)) +\
                      beta_pump_0 + beta_pump_1 / (T_chiller_water_return[t] - T_chiller_water_supply[t]) * ((T_out[t] -  quicksum(Chi[t] * Temp_Zone[i,t] for i in range(I_zone))) * v_out[t] * C_air +\
                      quicksum(Chi[i] * (Temp_Zone[i,t] - Temp_AC[i,t]) for i in range(I_zone)) * (v_sup[t] - v_out[t]) * C_air))
    
# constarint 2
for t in range(T):
    DCMG.addConstr(e_in[t] >= 0)

# constarint 3
DCMG.addConstr(Battery_state[0] == 0.5 * Battery_capacity)
   
# constarint 3.5
for t in range(T):
    DCMG.addConstr(Battery_state[t + 1] == Battery_state[t] + p_char[t] * eta_char_b - p_dis[t] * eta_dischar_b)
    DCMG.addConstr(Battery_state[t + 1] >= Battery_capacity_lowbound)
    DCMG.addConstr(Battery_state[t + 1] <= Battery_capacity_upbound)
    
    
# constarint 4 & 5
for t in range(T):
    DCMG.addConstr(p_char[t] <= char_b_upperbound * u_char[t])
    DCMG.addConstr(p_dis[t] <= dis_b_upperbound * u_dis[t])

# constarint 6
for t in range(T):
    DCMG.addConstr(u_char[t] + u_dis[t] <= 1)
    
# constarint 7 & 7.5 
for i in range(I_zone):
    #print(i // I_zone_length, i % I_zone_length)
    Temp_Zone[i,0] == T_zone_initial[i// I_zone_length, i % I_zone_length]  # unit: C
for i in range(I_zone):
    for t in range(T):
        Temp_Zone[i, t + 1] =  quicksum(adjacency * (Temp_Zone[node,t] - Temp_Zone[i,t])  for node, adjacency in enumerate(adjacent_zone_map[i,:]))/(C_air * R_air_ij) + \
        (m_air_in[i] * c_s_air[i] * (Temp_AC[i,t] - Temp_Zone[i,t]) + theta_heat[i,t]) / (C_air) 

# constarint 8 & 8.5
for t in range(1, T + 1):
    for i in range(I_zone):
        DCMG.addConstr(Temp_Zone[i,t] >= T_datacenter_lowerbound[i,t-1])
        DCMG.addConstr(Temp_Zone[i,t] <= T_datacenter_upperbound[i,t-1])
        
        
# constarint 9 & 9.5
for t in range(T):
    DCMG.addConstr(v_vent[t] + v_out[t] >= v_vent_lowerbound[t])
    
# constarint 10
for t in range(T):
    DCMG.addConstr(quicksum( x_tower[j,t] * M_chiller_rated[j] for j in range(J_chiller)) >= \
    ((T_out[t] -  quicksum(Chi[t] * Temp_Zone[i,t] for i in range(I_zone))) * v_out[t] * C_air + \
    quicksum(Chi[i] * (Temp_Zone[i,t] - Temp_AC[i,t]) for i in range(I_zone)) * (v_sup[t] - v_out[t]) * C_air)/ \
    (T_chiller_water_return[t] - T_chiller_water_supply[t]))
    
# constarint 11 
for t in range(T):
    DCMG.addConstr(quicksum( x_tower[j,t] * M_tower_rated[j] for j in range(J_tower)) >= \
    ((T_out[t] -  quicksum(Chi[t] * Temp_Zone[i,t] for i in range(I_zone))) * v_out[t] * C_air + \
    quicksum(Chi[i] * (Temp_Zone[i,t] - Temp_AC[i,t]) for i in range(I_zone)) * (v_sup[t] - v_out[t]) * C_air)/ \
    (T_tower_water_return[t] - T_tower_water_supply[t]))
    
# constarint 12(added)
for i in range(I_zone):
    for t in range(T):
        DCMG.addConstr(Temp_AC[i,t] >= T_AC_lowerbound)
        DCMG.addConstr(Temp_AC[i,t] <= T_AC_upperbound)

    

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [36]:
DCMG_sol = DCMG.optimize()
#assert DCMG_sol
DCMG.display()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads
Optimize a model with 189 rows, 162 columns and 1492 nonzeros
Model fingerprint: 0x42ceddfc
Variable types: 102 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e-13, 3e+07]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 118 rows and 98 columns
Presolve time: 0.00s
Presolved: 71 rows, 64 columns, 1030 nonzeros
Variable types: 62 continuous, 2 integer (0 binary)
Found heuristic solution: objective 181258.18318

Root relaxation: objective 1.682599e+05, 78 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

  